In [83]:
import notebook_common
import jaconv
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    # w_word_2400_without_guide_csv,
    # ------------------JLPT word------------------
    r_jlpt_csv,
    r_jlpt_v2_csv,
    w_jlpt_csv,
    w_jlpt_word_csv,
    w_jlpt_multiidx_agg_csv,
    r_jlpt_12000_csv,
    w_jlpt_12000_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    # w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    # w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    # w_merged_freq_sumed_csv,
    w_raw_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    re,
    l,
    fetch_random_from_set,
)

l("Import common module")

----------------------------------------Import common module----------------------------------------


In [84]:
# 读取原始 jlpt文件
df_jlpt_12000 = pd.read_csv(
    r_jlpt_12000_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    keep_default_na=False,
    # index_col=["word", "reading"]
)
df_jlpt_12000


,level,word,tone,ps,hira,meaning
0,NEW-JLPT::NEW-N5,〜やすい,,接尾,〜やすい,简单的，容易……的
1,NEW-JLPT::NEW-N5,〜か月,,接尾,〜かげつ,……个月
2,NEW-JLPT::NEW-N5,キログラム,③,名,(英) kilogram；(法) kilogramme,千克，公斤
3,NEW-JLPT::NEW-N5,キロメートル,,名,(法) kilomètre,……千米，……公里
4,NEW-JLPT::NEW-N5,グラム,①⓪,名,(英) gram；(法) gramme,克
...,...,...,...,...,...,...
12633,NEW-JLPT::NEW-N1::NEW-N1-核心,鎮める,③,他動2,しずめる,镇定，平息（混乱）；镇，止住（疼痛）
12634,NEW-JLPT::NEW-N3::NEW-N3-核心,付き,,名,つき,附着；燃烧；运气
12635,NEW-JLPT::NEW-N1::NEW-N1-核心,罰[ばち],②,名,ばち,报应
12636,NEW-JLPT::NEW-N2::NEW-N2-核心,潜[もぐ]る,②,自動1,もぐる,潜水；躲入，钻进


In [85]:
# 去除多余的~
def clean_title(val):
    if pd.isna(val) or val == "":
        return val
    return re.sub(r"^~|^〜|^～|^—|~$|～$|〜$", "", str(val))


# 将其~开头或结尾的值删除
df_jlpt_12000["word"] = df_jlpt_12000["word"].astype(str).apply(clean_title)
df_jlpt_12000["hira"] = df_jlpt_12000["hira"].astype(str).apply(clean_title)
df_jlpt_12000

,level,word,tone,ps,hira,meaning
0,NEW-JLPT::NEW-N5,やすい,,接尾,やすい,简单的，容易……的
1,NEW-JLPT::NEW-N5,か月,,接尾,かげつ,……个月
2,NEW-JLPT::NEW-N5,キログラム,③,名,(英) kilogram；(法) kilogramme,千克，公斤
3,NEW-JLPT::NEW-N5,キロメートル,,名,(法) kilomètre,……千米，……公里
4,NEW-JLPT::NEW-N5,グラム,①⓪,名,(英) gram；(法) gramme,克
...,...,...,...,...,...,...
12633,NEW-JLPT::NEW-N1::NEW-N1-核心,鎮める,③,他動2,しずめる,镇定，平息（混乱）；镇，止住（疼痛）
12634,NEW-JLPT::NEW-N3::NEW-N3-核心,付き,,名,つき,附着；燃烧；运气
12635,NEW-JLPT::NEW-N1::NEW-N1-核心,罰[ばち],②,名,ばち,报应
12636,NEW-JLPT::NEW-N2::NEW-N2-核心,潜[もぐ]る,②,自動1,もぐる,潜水；躲入，钻进


In [86]:
# 拆分 ・
split_df = df_jlpt_12000["ps"].str.split("・", n=3, expand=True)
df_jlpt_12000["prop1"] = split_df[0].fillna("")
df_jlpt_12000["prop2"] = split_df[1].fillna("")
df_jlpt_12000["prop3"] = split_df[2].fillna("")
df_jlpt_12000["prop4"] = split_df[3].fillna("")
to_csv_sig(df_jlpt_12000,w_jlpt_12000_csv)
l('df_jlpt_12000')
df_jlpt_12000

----------------------------------------df_jlpt_12000----------------------------------------


,level,word,tone,ps,hira,meaning,prop1,prop2,prop3,prop4
0,NEW-JLPT::NEW-N5,やすい,,接尾,やすい,简单的，容易……的,接尾,,,
1,NEW-JLPT::NEW-N5,か月,,接尾,かげつ,……个月,接尾,,,
2,NEW-JLPT::NEW-N5,キログラム,③,名,(英) kilogram；(法) kilogramme,千克，公斤,名,,,
3,NEW-JLPT::NEW-N5,キロメートル,,名,(法) kilomètre,……千米，……公里,名,,,
4,NEW-JLPT::NEW-N5,グラム,①⓪,名,(英) gram；(法) gramme,克,名,,,
...,...,...,...,...,...,...,...,...,...,...
12633,NEW-JLPT::NEW-N1::NEW-N1-核心,鎮める,③,他動2,しずめる,镇定，平息（混乱）；镇，止住（疼痛）,他動2,,,
12634,NEW-JLPT::NEW-N3::NEW-N3-核心,付き,,名,つき,附着；燃烧；运气,名,,,
12635,NEW-JLPT::NEW-N1::NEW-N1-核心,罰[ばち],②,名,ばち,报应,名,,,
12636,NEW-JLPT::NEW-N2::NEW-N2-核心,潜[もぐ]る,②,自動1,もぐる,潜水；躲入，钻进,自動1,,,


In [87]:
# 处理自 他
# 处理A/B/C/D列并生成对应X1列
for col in ["prop1", "prop2", "prop3", "prop4"]:
    # 提取前缀和动后部分
    extracted = df_jlpt_12000[col].str.extract(r"^(他|自|自他)(動[1-3].*)$")

    # 更新原列内容
    df_jlpt_12000[col] = np.where(
        extracted[1].notna(),
        extracted[1],
        df_jlpt_12000[col],  # 匹配成功时取动后部分  # 匹配失败时保留原值
    )

    # 新增X1列
    df_jlpt_12000[f"{col}_z"] = extracted[0].fillna("")
# 生成X列：取第一个非空的X1列
df_jlpt_12000["type"] = df_jlpt_12000[["prop1_z", "prop2_z", "prop3_z", "prop4_z"]].apply(
    lambda row: next((val for val in row if val != ""), ""), axis=1
)
to_csv_sig(df_jlpt_12000, w_jlpt_12000_csv)
l("df_jlpt_12000")
df_jlpt_12000

----------------------------------------df_jlpt_12000----------------------------------------


,level,word,tone,ps,hira,meaning,prop1,prop2,prop3,prop4,prop1_z,prop2_z,prop3_z,prop4_z,type
0,NEW-JLPT::NEW-N5,やすい,,接尾,やすい,简单的，容易……的,接尾,,,,,,,,
1,NEW-JLPT::NEW-N5,か月,,接尾,かげつ,……个月,接尾,,,,,,,,
2,NEW-JLPT::NEW-N5,キログラム,③,名,(英) kilogram；(法) kilogramme,千克，公斤,名,,,,,,,,
3,NEW-JLPT::NEW-N5,キロメートル,,名,(法) kilomètre,……千米，……公里,名,,,,,,,,
4,NEW-JLPT::NEW-N5,グラム,①⓪,名,(英) gram；(法) gramme,克,名,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12633,NEW-JLPT::NEW-N1::NEW-N1-核心,鎮める,③,他動2,しずめる,镇定，平息（混乱）；镇，止住（疼痛）,動2,,,,他,,,,他
12634,NEW-JLPT::NEW-N3::NEW-N3-核心,付き,,名,つき,附着；燃烧；运气,名,,,,,,,,
12635,NEW-JLPT::NEW-N1::NEW-N1-核心,罰[ばち],②,名,ばち,报应,名,,,,,,,,
12636,NEW-JLPT::NEW-N2::NEW-N2-核心,潜[もぐ]る,②,自動1,もぐる,潜水；躲入，钻进,動1,,,,自,,,,自


In [88]:
# 字符串替换
replace_dict = {
    "動1": "动1",
    "接頭": "接头词",
    "接尾": "接尾词",
    "連体": "连体词",
    "名名": "名词",
    "接": "连词",
    "イ形": "形1",
    "ナ形": "形2",
    "動2": "动2",
    "動3": "动3",
    "代": "代词",
    "名": "名词",
    "感": "感叹词",
    "副": "副词",
}
cols = ["prop1", "prop2", "prop3", "prop4"]
df_jlpt_12000[cols] = df_jlpt_12000[cols].replace(replace_dict)
#
df_jlpt_12000["remark"] = ""  # 先初始化为空
df_jlpt_12000.loc[df_jlpt_12000["hira"].str.startswith("("), "remark"] = df_jlpt_12000[
    "hira"
]

df_jlpt_12000["hira"] = np.where(
    df_jlpt_12000["hira"].str.startswith("("),
    df_jlpt_12000["word"].apply(jaconv.kata2hira),
    df_jlpt_12000["hira"],
)

to_csv_sig(df_jlpt_12000, w_jlpt_12000_csv)
l("df_jlpt_12000")
df_jlpt_12000

----------------------------------------df_jlpt_12000----------------------------------------


,level,word,tone,ps,hira,meaning,prop1,prop2,prop3,prop4,prop1_z,prop2_z,prop3_z,prop4_z,type,remark
0,NEW-JLPT::NEW-N5,やすい,,接尾,やすい,简单的，容易……的,接尾词,,,,,,,,,
1,NEW-JLPT::NEW-N5,か月,,接尾,かげつ,……个月,接尾词,,,,,,,,,
2,NEW-JLPT::NEW-N5,キログラム,③,名,きろぐらむ,千克，公斤,名词,,,,,,,,,(英) kilogram；(法) kilogramme
3,NEW-JLPT::NEW-N5,キロメートル,,名,きろめーとる,……千米，……公里,名词,,,,,,,,,(法) kilomètre
4,NEW-JLPT::NEW-N5,グラム,①⓪,名,ぐらむ,克,名词,,,,,,,,,(英) gram；(法) gramme
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12633,NEW-JLPT::NEW-N1::NEW-N1-核心,鎮める,③,他動2,しずめる,镇定，平息（混乱）；镇，止住（疼痛）,动2,,,,他,,,,他,
12634,NEW-JLPT::NEW-N3::NEW-N3-核心,付き,,名,つき,附着；燃烧；运气,名词,,,,,,,,,
12635,NEW-JLPT::NEW-N1::NEW-N1-核心,罰[ばち],②,名,ばち,报应,名词,,,,,,,,,
12636,NEW-JLPT::NEW-N2::NEW-N2-核心,潜[もぐ]る,②,自動1,もぐる,潜水；躲入，钻进,动1,,,,自,,,,自,


In [ ]:
#查询 2400词汇在 12000里的比例




